In [ ]:
# Grad-CAM for 50 Random Tampered Images

import os
import random
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

class TamperCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(128 * 16 * 16, 128), nn.ReLU(), nn.Dropout(0.5),
            nn.Linear(128, 1), nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x).squeeze()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TamperCNN()
model.load_state_dict(torch.load("tampered_patch_cnn.pth", map_location=device))
model.to(device).eval()

activations = None
gradients = None

def save_activation(module, input, output):
    global activations
    activations = output

def save_gradient(module, grad_input, grad_output):
    global gradients
    gradients = grad_output[0]

model.net[6].register_forward_hook(save_activation)
model.net[6].register_full_backward_hook(save_gradient)

folder_path = r"C:\Users\imran\OneDrive\Robotics Projects\Image Forgery\Image Data\VALIDATION_CG-1050\VALIDATION\TAMPERED"

all_images = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.lower().endswith(('.jpg', '.png'))]
random_images = random.sample(all_images, 50)

def generate_gradcam(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (128, 128))
    img_np = img / 255.0
    img_tensor = torch.tensor(img_np, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)
    img_tensor.requires_grad_()

    output = model(img_tensor)
    prediction = output.item()
    model.zero_grad()
    output.backward()

    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    activation_map = activations.squeeze()

    for i in range(activation_map.shape[0]):
        activation_map[i] *= pooled_gradients[i]

    heatmap = torch.mean(activation_map, dim=0).cpu().detach().numpy()
    heatmap = np.maximum(heatmap, 0)
    heatmap = cv2.resize(heatmap, (128, 128))
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)

    img_uint8 = (img_np * 255).astype(np.uint8)
    heatmap_color = cv2.applyColorMap(np.uint8(255 * heatmap), cv2.COLORMAP_JET)
    overlay = cv2.addWeighted(img_uint8, 0.6, heatmap_color, 0.4, 0)

    return img_uint8, heatmap, overlay, prediction

for i, path in enumerate(random_images):
    original, heatmap, overlay, pred = generate_gradcam(path)
    fname = os.path.basename(path)

    plt.figure(figsize=(12, 4))
    plt.suptitle(f"Random Image {i+1}: {fname} | Predicted: {'Tampered' if pred > 0.5 else 'Original'}", fontsize=12)

    plt.subplot(1, 3, 1)
    plt.imshow(original)
    plt.title("Original Patch")
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(heatmap, cmap='jet')
    plt.title("Grad-CAM Heatmap")
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(overlay)
    plt.title("Overlay")
    plt.axis('off')

    plt.tight_layout()
    plt.show()
